Purpose: Function to accept user-defined LEHD input file from the FTP site and a local csv of ACS data (this script is built to use the same format as the ACS csv download from CO Demography office site)

In [1]:
# Import necessary libraries
import pandas as pd

In [13]:
# Define function to retrieve and read the file from the site
def aggregate_LEHD(_state, _type, _segment, _JT_code, _year):
    urlbase = 'https://lehd.ces.census.gov/data/lodes/LODES7/'
    file_name = _state + '_' + _type + '_' + _segment + '_' + _JT_code + '_' + _year + '.csv.gz'
    file_path = urlbase + _state + '/' + _type + '/' 
    file = pd.read_csv(file_path + file_name)
    #Clip the wi_rac dataframe to include only the relevant columns: C000, CE01, CE02, CE03, h_geocode
    #wi_rac2 = wi_rac.iloc['h_geocode','C000','CE01','CE02','CE03'],[]
    file_clipped = file.iloc[:,[0,1,5,6,7]]
    # Add a column that will later hold the block group number from each h_code
    file_clipped.insert(1, 'BlkGrp_code', '0')
    # Set block group equal to the string h_geocode
    file_clipped.loc[:,'BlkGrp_code']= file_clipped.loc[:,'h_geocode'].astype(str)
    # Chop the string down to the countytractblockgroup numbers only, cutting off the block id (last 3 digits).
    file_clipped.loc[:,'BlkGrp_code']= file_clipped.loc[:,'BlkGrp_code'].str[:-3]
    # For each unique BlkGrp_code value, aggregate the rows into one representing the sum of each jobs column while taking
    # the first h_geocode to make sure the blk grp code is correct
    file_agg = file_clipped.groupby(file_clipped['BlkGrp_code']).agg({'h_geocode':'first', 'C000':'sum', 'CE01':'sum','CE02':'sum','CE03':'sum',})
    #Add columns containing information on percent low, me, high income jobs
    file_agg['CE01%']= file_agg['CE01']/file_agg['C000']
    file_agg['CE02%']= file_agg['CE02']/file_agg['C000']
    file_agg['CE03%']= file_agg['CE03']/file_agg['C000']
    
    return file_agg

In [18]:
# Define function to read in and prep ACS csv for merging to LEHD:
def get_ACS(_ACS_file):
    ACS = pd.read_csv(_ACS_file)
    # Drop unnecessary columns that come default with CO Demography office export
    ACS = ACS.drop(['geoname','state','county','place','tract','bg'], axis= 1)
    ACS = ACS.iloc[1:,:]
    # Set geonum column to type string
    ACS.loc[:,'geonum']= ACS.loc[:,'geonum'].astype(str)
    # Chop the string down to the countytractblockgroup numbers only, cutting off the unnecessary leading 1.
    ACS.loc[:,'geonum']= ACS.loc[:,'geonum'].str[1:]
    
    return ACS

In [23]:
def main(_state, _type, _segment, _JT_code, _year, _ACS_file):
    rac_file = aggregate_LEHD(_state,_type,_segment,_JT_code,_year)
    ACS_file = get_ACS(_ACS_file)
    merged = rac_file.merge(ACS_file, left_on= 'BlkGrp_code', right_on= 'geonum')
    return merged

In [24]:
wi_test = main('wi', 'rac', 'S000', 'JT00', '2010', 'WI_ACS_2009_2013_Population.csv')

In [25]:
print(wi_test[:5])

         h_geocode  C000  CE01  CE02  CE03     CE01%     CE02%     CE03%  \
0  550019501001000   419   144   149   126  0.343675  0.355609  0.300716   
1  550019501002000   388   120   140   128  0.309278  0.360825  0.329897   
2  550019501003001   402   149   120   133  0.370647  0.298507  0.330846   
3  550019502011000   525   184   215   126  0.350476  0.409524  0.240000   
4  550019502021000   277    86   121    70  0.310469  0.436823  0.252708   

         geonum b01003001  b01003_moe001  
0  550019501001      1352          248.0  
1  550019501002       663          195.0  
2  550019501003      1026          154.0  
3  550019502011      1426          136.0  
4  550019502021       788          102.0  
